# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f2e4cc22fa0>
├── 'a' --> tensor([[ 0.3878, -0.6174,  0.5047],
│                   [ 0.3918,  0.7357, -0.3302]])
└── 'x' --> <FastTreeValue 0x7f2e4cc15220>
    └── 'c' --> tensor([[-0.1744, -1.9221,  0.5118,  0.4322],
                        [ 0.8708, -0.2826, -1.7390,  1.3074],
                        [ 1.9699, -0.1494,  0.3028, -1.6403]])

In [4]:
t.a

tensor([[ 0.3878, -0.6174,  0.5047],
        [ 0.3918,  0.7357, -0.3302]])

In [5]:
%timeit t.a

67.7 ns ± 0.0667 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f2e4cc22fa0>
├── 'a' --> tensor([[-1.6727, -0.3736, -1.7035],
│                   [ 0.6524, -0.0887, -0.5390]])
└── 'x' --> <FastTreeValue 0x7f2e4cc15220>
    └── 'c' --> tensor([[-0.1744, -1.9221,  0.5118,  0.4322],
                        [ 0.8708, -0.2826, -1.7390,  1.3074],
                        [ 1.9699, -0.1494,  0.3028, -1.6403]])

In [7]:
%timeit t.a = new_value

70.3 ns ± 0.257 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.3878, -0.6174,  0.5047],
               [ 0.3918,  0.7357, -0.3302]]),
    x: Batch(
           c: tensor([[-0.1744, -1.9221,  0.5118,  0.4322],
                      [ 0.8708, -0.2826, -1.7390,  1.3074],
                      [ 1.9699, -0.1494,  0.3028, -1.6403]]),
       ),
)

In [10]:
b.a

tensor([[ 0.3878, -0.6174,  0.5047],
        [ 0.3918,  0.7357, -0.3302]])

In [11]:
%timeit b.a

60.3 ns ± 0.0173 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.4766, -1.2001, -0.9965],
               [-1.0203,  0.8614,  0.6661]]),
    x: Batch(
           c: tensor([[-0.1744, -1.9221,  0.5118,  0.4322],
                      [ 0.8708, -0.2826, -1.7390,  1.3074],
                      [ 1.9699, -0.1494,  0.3028, -1.6403]]),
       ),
)

In [13]:
%timeit b.a = new_value

493 ns ± 0.188 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

821 ns ± 0.954 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 89 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

144 µs ± 338 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 609 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f2d9f88b790>
├── 'a' --> tensor([[[ 0.3878, -0.6174,  0.5047],
│                    [ 0.3918,  0.7357, -0.3302]],
│           
│                   [[ 0.3878, -0.6174,  0.5047],
│                    [ 0.3918,  0.7357, -0.3302]],
│           
│                   [[ 0.3878, -0.6174,  0.5047],
│                    [ 0.3918,  0.7357, -0.3302]],
│           
│                   [[ 0.3878, -0.6174,  0.5047],
│                    [ 0.3918,  0.7357, -0.3302]],
│           
│                   [[ 0.3878, -0.6174,  0.5047],
│                    [ 0.3918,  0.7357, -0.3302]],
│           
│                   [[ 0.3878, -0.6174,  0.5047],
│                    [ 0.3918,  0.7357, -0.3302]],
│           
│                   [[ 0.3878, -0.6174,  0.5047],
│                    [ 0.3918,  0.7357, -0.3302]],
│           
│                   [[ 0.3878, -0.6174,  0.5047],
│                    [ 0.3918,  0.7357, -0.3302]]])
└── 'x' --> <FastTreeValue 0x7f2d9f88b130>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.2 µs ± 47.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2e4cc8fe50>
├── 'a' --> tensor([[ 0.3878, -0.6174,  0.5047],
│                   [ 0.3918,  0.7357, -0.3302],
│                   [ 0.3878, -0.6174,  0.5047],
│                   [ 0.3918,  0.7357, -0.3302],
│                   [ 0.3878, -0.6174,  0.5047],
│                   [ 0.3918,  0.7357, -0.3302],
│                   [ 0.3878, -0.6174,  0.5047],
│                   [ 0.3918,  0.7357, -0.3302],
│                   [ 0.3878, -0.6174,  0.5047],
│                   [ 0.3918,  0.7357, -0.3302],
│                   [ 0.3878, -0.6174,  0.5047],
│                   [ 0.3918,  0.7357, -0.3302],
│                   [ 0.3878, -0.6174,  0.5047],
│                   [ 0.3918,  0.7357, -0.3302],
│                   [ 0.3878, -0.6174,  0.5047],
│                   [ 0.3918,  0.7357, -0.3302]])
└── 'x' --> <FastTreeValue 0x7f2e4cc8fa30>
    └── 'c' --> tensor([[-0.1744, -1.9221,  0.5118,  0.4322],
                        [ 0.8708, -0.2826, -1.7390,  1.3074],
                 

In [23]:
%timeit t_cat(trees)

29.8 µs ± 59.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

55.9 µs ± 113 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.1744, -1.9221,  0.5118,  0.4322],
                       [ 0.8708, -0.2826, -1.7390,  1.3074],
                       [ 1.9699, -0.1494,  0.3028, -1.6403]],
              
                      [[-0.1744, -1.9221,  0.5118,  0.4322],
                       [ 0.8708, -0.2826, -1.7390,  1.3074],
                       [ 1.9699, -0.1494,  0.3028, -1.6403]],
              
                      [[-0.1744, -1.9221,  0.5118,  0.4322],
                       [ 0.8708, -0.2826, -1.7390,  1.3074],
                       [ 1.9699, -0.1494,  0.3028, -1.6403]],
              
                      [[-0.1744, -1.9221,  0.5118,  0.4322],
                       [ 0.8708, -0.2826, -1.7390,  1.3074],
                       [ 1.9699, -0.1494,  0.3028, -1.6403]],
              
                      [[-0.1744, -1.9221,  0.5118,  0.4322],
                       [ 0.8708, -0.2826, -1.7390,  1.3074],
                       [ 1.9699, -0.1494,  0.3028, -1.6403]],

In [26]:
%timeit Batch.stack(batches)

77.2 µs ± 136 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.1744, -1.9221,  0.5118,  0.4322],
                      [ 0.8708, -0.2826, -1.7390,  1.3074],
                      [ 1.9699, -0.1494,  0.3028, -1.6403],
                      [-0.1744, -1.9221,  0.5118,  0.4322],
                      [ 0.8708, -0.2826, -1.7390,  1.3074],
                      [ 1.9699, -0.1494,  0.3028, -1.6403],
                      [-0.1744, -1.9221,  0.5118,  0.4322],
                      [ 0.8708, -0.2826, -1.7390,  1.3074],
                      [ 1.9699, -0.1494,  0.3028, -1.6403],
                      [-0.1744, -1.9221,  0.5118,  0.4322],
                      [ 0.8708, -0.2826, -1.7390,  1.3074],
                      [ 1.9699, -0.1494,  0.3028, -1.6403],
                      [-0.1744, -1.9221,  0.5118,  0.4322],
                      [ 0.8708, -0.2826, -1.7390,  1.3074],
                      [ 1.9699, -0.1494,  0.3028, -1.6403],
                      [-0.1744, -1.9221,  0.5118,  0.4322],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 450 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

322 µs ± 915 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
